### SageMaker chemprot prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


In [2]:
version_tag="202207311958"
pytorch_custom_image_name=f"large-scale-ptm-ppi:gpu-{version_tag}"
instance_type = "ml.g4dn.2xlarge"  #ml.g4dn.2xlarge
instance_count = 3



In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [4]:
bucket = "aegovan-data"

In [5]:

sentence_trainfile = "s3://{}/chemprot/chemprot_train.json".format(bucket)
sentence_testfile= "s3://{}/chemprot/chemprot_test.json".format(bucket)
sentence_valfile="s3://{}/chemprot/chemprot_dev.json".format(bucket)

abstract_testfile= "s3://{}/chemprot/chemprot_abstract_test.json".format(bucket)
abstract_trainfile= "s3://{}/chemprot/chemprot_abstract_train.json".format(bucket)
abstract_valfile= "s3://{}/chemprot/chemprot_abstract_val.json".format(bucket)

abstract_largescale = "s3://{}/chemprotlargescale/input/data_2022080618".format(bucket)

eval_file = abstract_largescale

In [6]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H")

In [7]:
#training_job = "chemprot-bert-f1-2022-05-22-23-18-21-560"
training_job="chemprot-abstract-bert-f1-2022-07-30-22-34-07-969"

In [8]:
s3_model_path = f"s3://aegovan-data/chemprot_sagemakerresults/{training_job}/output/model.tar.gz"



### Run  prediction

In [9]:
#s3_output_predictions = "s3://aegovan-data/pubmed_asbtract/predictions_largescale_{}_{}/".format(job_prefix,date_fmt)
s3_output_predictions = "s3://aegovan-data/chemprot/predictions_{}/{}".format(training_job,date_fmt)
s3_input_data = eval_file
s3_data_type="S3Prefix"
usefilter=0
filter_threshold_std=1.0

s3_input_models = s3_model_path
s3_input_vocab = "s3://{}/embeddings/bert/".format(bucket)

In [10]:
s3_input_data, s3_data_type

('s3://aegovan-data/chemprot/chemprot_abstract_train.json', 'S3Prefix')

In [11]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(image_uri=docker_repo,
                                       command=["python"],
                                       env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                       role=role,
                                       instance_type=instance_type,
                                       instance_count=instance_count,
                                       max_runtime_in_seconds= 5 * 24 * 60 * 60,
                                       volume_size_in_gb = 200,
                                       network_config=NetworkConfig(enable_network_isolation=False),
                                       base_job_name ="chemprot-inference"
                                       )


sm_local_input_models = "/opt/ml/processing/input/data/models"
sm_local_input_data = "/opt/ml/processing/input/data/jsondata"
sm_local_input_vocab = "/opt/ml/processing/input/data/vocab"


sm_local_output = "/opt/ml/processing/output"

input_file_name = s3_input_data.split("/")[-1]

script_processor.run(
        code='../src/inference/chemprot_batch_predict.py',

        arguments=[
            sm_local_input_data,
            sm_local_input_models,
            sm_local_output,
            "--ensemble", "0",
            "--tokenisor_data_dir", sm_local_input_vocab,           
            "--filter", str(usefilter),
            "--batch", "32",
            "--filterstdthreshold", str(filter_threshold_std)
        ],

        inputs=[
                ProcessingInput(
                    source=s3_input_data,
                    s3_data_type = s3_data_type,
                    destination=sm_local_input_data,
                    s3_data_distribution_type="ShardedByS3Key"),

            ProcessingInput(
                    source=s3_input_models,
                    destination=sm_local_input_models,
                    s3_data_distribution_type="FullyReplicated"),
            
            ProcessingInput(
                    source=s3_input_vocab,
                    destination=sm_local_input_vocab,
                    s3_data_distribution_type="FullyReplicated")
            ],


        outputs=[ProcessingOutput(
                source=sm_local_output, 
                destination=s3_output_predictions,
                output_name='predictions')]
    )





Job Name:  chemprot-inference-2022-08-07-00-55-54-399
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/chemprot/chemprot_abstract_train.json', 'LocalPath': '/opt/ml/processing/input/data/jsonlines', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/chemprot_sagemakerresults/chemprot-abstract-bert-f1-2022-07-30-22-34-07-969/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/input/data/models', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aegovan-data/embeddings/bert/', 'LocalPath': '/opt/ml/processing/input/data/vocab', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionT